In [1]:
from keras.layers import Input, Dense, TimeDistributed, Embedding
from keras.layers import Concatenate, Reshape, Lambda, Multiply, multiply, concatenate
from keras.models import Model
from keras import backend as K

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import tensorflow as tf
import numpy as np

from dataset_load import *
from human_terms_network import *

C:\Users\Anneke\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load

In [2]:
path = r"..\..\data\reviews_Amazon_Instant_Video_5.json.gz"

X, y = extract_review_amazon(path, 'reviewText')

In [3]:
y_label = np.asarray(y)

y_label = np.ndarray.astype(y_label>2, dtype='int32')
y_label

array([0, 1, 0, ..., 1, 1, 1])

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, ShuffleSplit

# split
X_train_split, X_test_split, y_train, y_test = train_test_split(X, y_label, test_size=0.33, random_state=42)

# preprocessing
X_train_corpus_update = update_corpus_contraction(X_train_split)
X_test_corpus_update = update_corpus_contraction(X_test_split)

(75, 2)
corpus update start
corpus update end

(75, 2)
corpus update start
corpus update end



In [8]:
# count vectorizer
token = r"(?u)\b[\w\'/]+\b"
cv = CountVectorizer(lowercase=True, max_df=1.0, min_df=100, binary=True, token_pattern=token)
cv.set_params(ngram_range=(1,1))

cv.fit(X_train_split)

X_train = cv.transform(X_train_corpus_update)
X_test = cv.transform(X_test_corpus_update)

words = cv.get_feature_names()

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42)

clf.fit(X_train, y_train)

print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

In [ ]:
weight = clf.coef_.flatten()
indices = np.argsort(weight)[::-1]

for i in indices[:50]:
    print("%s \t %.3f" %(words[i], weight[i]))

reversed_indices = indices[::-1]
print()
for i in reversed_indices[:50]:
    print("%s \t %.3f" %(words[i], weight[i]))

In [ ]:
htm = Human_Terms_Network(input_shape=30, human_terms_shape=50)

# Plan
# Set the data
# including the y_agreement